<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/01-pyAMPACT_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Introduction</h1>

In [1]:
!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git
from IPython.utils import io
print('Importing libraries...')
with io.capture_output() as captured:
    !pip install --upgrade pandas
    !pip install -i https://test.pypi.org/simple/ --no-deps pyampact
    import pyampact
import pandas as pd
import numpy as np
import librosa

Cloning into 'pyAMPACTtutorials'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 417 (delta 146), reused 179 (delta 105), pack-reused 177
Receiving objects: 100% (417/417), 19.44 MiB | 12.14 MiB/s, done.
Resolving deltas: 100% (207/207), done.
Importing libraries...


Load audio with librosa and symbolic files with pyAMPACT's `Score` function. Audio files can be any format supported by librosa (wav, mp3, flac, ogg, etc.) and symbolic files can be any format supported by music21 (kern, xml, mei, midi, abc, etc.).

In [2]:
audio_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.wav'
score_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.krn'

y, original_sr = librosa.load(audio_file)

piece = pyampact.Score(score_file)

Run the alignment with pyampact's `run_alignment` function.



In [3]:
# You can specify the target sample rate (target_sr), window size in
# milliseconds (win_ms), the hop size (hop_length) as well as arguments
# that dictate the way that the symbolic file is represented as a
# spectrogram-like mask: the bin width of each note/harmonic (width)
# and the number of harmonics (n_harm).
target_sr = 4000
win_ms = 100
hop_length = 32
width = 3
n_harm = 3

# Visualize the alignment with True flag

res, dtw, spec, nmat = pyampact.run_alignment(
    y, original_sr, piece, piece.nmats(), width, target_sr, n_harm, win_ms, hop_length)

Estimate performance parameters.

In [4]:
# This is just calculating a single F0 for each frame, but separate F0s need to be calculated
# for each polyphonic note (as per https://github.com/jcdevaney/AMPACT-working/blob/main/f0EstWeightedSumSpec.m)
# A spectrum also need to be calculated and return for estimating timbral parameters
# The NMAT looped through and the onset, offset, and MIDI note passed to the calculate_f0_est function
# [f0{v}{loc}, pwr{v}{loc}, t{v}{loc}, M{v}{loc}, xf{v}{loc}] = f0EstWeightedSumSpec(audiofile, ons(loc), offs(loc), nmat{v}(n,4));
# In the original MATLAB code there was a separate NMAT for each part/voice (v) this can be done by creating a dictionary
# out of the nmat
#   nmat_dict = piece.nmats()
#   nmat_dict['Part-1']

def f0EstWeightedSumSpecWrapper(audio_file, hop_length, win_ms, target_sr):
    freqs, times, mags, f0_values, mags_mat = pyampact.ifgram(audiofile=audio_file, tsr=target_sr, win_ms=win_ms)
    mags_db = librosa.amplitude_to_db(mags, ref=np.max)
    f0_values, sig_pwr = pyampact.calculate_f0_est(audio_file, hop_length, win_ms, target_sr)
    sig_pwr = mags ** 2 # power of signal, magnitude/amplitude squared
    # Prune NaN and zero values from estimated F0 and power
    f0_values = f0_values[~np.isnan(f0_values)]
    sig_pwr = sig_pwr[sig_pwr != 0]

    return f0_values, sig_pwr, mags_mat

f0_values, sig_pwr, mags_mat = f0EstWeightedSumSpecWrapper(audio_file, hop_length, win_ms, target_sr)

Compile estimated performance parameters

In [5]:
performData = pyampact.data_compilation(f0_values, sig_pwr, mags_mat, nmat, target_sr, hop_length, y, audio_file)

In [6]:
performData[1]

{'Part-1': {'133339131146064': {'MEASURE': 1,
   'ONSET': 0.0,
   'DURATION': 2.0,
   'PART': 'Part-1',
   'MIDI': 79.0,
   'ONSET_SEC': 0.0,
   'OFFSET_SEC': 2.0,
   'ppitch1': 527.8350515463917,
   'ppitch2': 508.20512820512823,
   'jitter': 44.22764227642276,
   'vibratoDepth': 194.89614429807847,
   'vibratoRate': 1.0080645161290323,
   'pwrVals': '[ -7.455658  -23.249374  -14.911608  -20.111244  -15.503868  -12.258648\n  -4.8107934  -8.449371  -19.985888  -12.204973  -16.529787  -18.187176\n -41.17571   -10.2039995  -8.460949  -36.62087   -10.390907  -18.380072\n -13.442754  -20.057644  -15.734764  -18.739964  -17.393124  -18.199966\n -27.572859  -26.304188  -29.529543  -26.512022  -28.730947  -20.439808\n -19.189964  -41.14382   -14.703243   -8.21903   -10.324436  -25.122856\n -14.12919    -8.392443   -9.7932825 -17.306013  -14.914107   -8.821705\n -13.830166  -11.704625   -9.089252  -29.118084   -9.905396   -9.433115\n -21.495918  -13.765352  -18.46326    -8.462563  -18.48337   

Export estimated performance parameters to a kern or MEI file

In [7]:
piece.toKern()

'!!!COM: Mozart\n!!!OTL: Zwolf Variationen in C uber ein Menuett von Johann Christian Fischer\n**kern\t**kern\n*part2\t*part1\n*staff2\t*staff1\n*Ivox\t*Ivox\n*I"Part-2\t*I"Part-1\n*I\'P\t*I\'P\n=1\t=1\n*clefF4\t*clefG2\n*k[]\t*k[]\n*M3/4\t*M3/4\n4C\t2gg\n4E\t.\n4C\t16ffLL\n.\t16ee\n.\t16dd\n.\t16ccJJ\n=2\t=2\n4D\t8ggL\n.\t8ffJ\n4BB\t4.dd\n4GG\t.\n.\t8ff\n=3\t=3\n8DL\t12eeL\n.\t12gg\n8FJ\t.\n.\t12aaJ\n4G\t4cc\n4GG\t4b\n=4\t=4\n4C\t4b\n4GG\t4cc\n4CC\t4r\n==\t==\n*-\t*-\n!!!RDF**kern: %=rational rhythm\n!!!RDF**kern: l=long note in original notation\n!!!RDF**kern: i=editorial accidental\n!!!ONB: Translated from a krn file on 2024-04-09 via pyAMPACT\n!!!title: @{OTL}'

In [8]:
piece.toMEI()